#### Bias only model 
 - Write down code for the gradient equations
 - Use gradient descent library to optimize the model

In [1]:
import gzip
from collections import defaultdict
import random
import numpy
import scipy.optimize
import scipy

In [2]:
path = "/Users/cnogueira/Documents/Development/Notebooks/study/week2/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"

In [3]:
f = gzip.open(path, 'rt', encoding="utf8")
header = f.readline()

In [4]:
header = header.strip().split('\t')

In [5]:
dataset = []
for line in f: 
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)

In [6]:
# Here i am creating two data structures:  
#For each user, which itens did they consume (users per item), and
#For each items, which users consumed that item (item per user)
usersPerItem = defaultdict(set) #Ui
itemsPerUser = defaultdict(set) #Iu
itemNames = {}
for d in dataset:
    user, item = d['customer_id'], d['product_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    itemNames[item] = d['product_title']

In [7]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
for d in dataset:
    user, item = d['customer_id'], d['product_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [8]:
N = len(dataset)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

In [9]:
# This is just to see if the algorithm is better to just predict the mean in terms of mean square error all the time
ratingMean = sum([d['star_rating'] for d in dataset])/len(dataset)

### Fitting
    - Alpha and Betau/Betai (userbiases) are the parameters I will fit.  

In [10]:
# Initial value of alpha is the mean reating to help the algo to convert qickly
alpha = ratingMean

In [11]:
# Initial valie of betai and betau is zerp
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [12]:
# Each user u have a vector of parameters saying "What are the preference dimensions for that user u?"
# Each item i have a vector of parameters saying "What are the properties of those items?"
userGamma = {}
itemGamma = {}

In [13]:
# set latent factors dimensionality to be 2 (simple model)
K = 2

In [14]:
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [15]:
for i in reviewsPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [16]:
def inner(x, y):
    return sum([a*b for a,b in zip(x, y)])

In [17]:
# The prediction function is just the bias only model
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])

In [18]:
# Unpack function

def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[0] # Alpha is the first position on the vector
    index = index + 1
    userBiases = dict(zip(users, theta[index:nUsers+index])) 
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index+= nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index +=K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index+= K

In [19]:
# Cost function is also required by the library (is written on the notes)
def cost(theta, labels, lamb):
    unpack(theta) # extract alpha, betai, betau
    # Use those values to make predictions in the datapoints on the training set
    predictions = [prediction(d['customer_id'], d['product_id']) for d in dataset]
    cost = MSE(predictions, labels)
    # Debugging
    print("MSE = " + str(cost))
    # Regularization
    for u in users:
        cost *= lamb*userBiases[u]**2
        for k in range(K):
            cost +=lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

In [37]:
# Derivative function has a corresponding derivative term for each parameter
def derivative(theta, labels, lamb):
    unpack(theta) # have alpha, betau for each user and betai for each item
    N =  len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in reviewsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in reviewsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for d in dataset:
        u,i = d['customer_id'], d['product_id']
        pred = prediction(u, i)
        diff = pred - d['star_rating']
        dalpha += 2/N*diff
        dUserBiases[u] +=  2/N*diff
        dItemBiases[u] +=  2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases: 
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2/N*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2/N*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta)

In [38]:
# mean square error
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions, labels)]
    return sum(differences)/len(differences)
# baseline
alwaysPredictMean = [ratingMean for d in dataset]


In [39]:
labels = [d['star_rating'] for d in dataset]
# Compute the accuracty of trivial baseline
MSE(alwaysPredictMean, labels)

1.4796142779564334

In [40]:
scipy.optimize.fmin_l_bfgs_b(cost, [ratingMean] + [0.0]* (nUsers*(K+1)+nItems*(K+1)),
                            derivative, args = (labels, 0.001))

MSE = 1.4796142779564334
MSE = 1.4764397325863097
MSE = 1.4787141438375289
MSE = 1.4794090186282771
MSE = 1.479570207519254
MSE = 1.4796049432028695
MSE = 1.479612306458805
MSE = 1.4796138618447625
MSE = 1.4796141901407405
MSE = 1.4796142594298556
MSE = 1.4796142740506226
MSE = 1.4796142771472431
MSE = 1.4796142778005967
MSE = 1.4796142779238735
MSE = 1.479614277945896
MSE = 1.4796142779539028
MSE = 1.4796142779559502
MSE = 1.47961427795641
MSE = 1.479614277956431
MSE = 1.4796142779564334
MSE = 1.4796142779564334


(array([4.25110277, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 0.0,
 {'grad': array([ 8.33042349e-13,  2.76558614e-06, -1.15881596e-05, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
  'task': b'ABNORMAL_TERMINATION_IN_LNSRCH',
  'funcalls': 21,
  'nit': 0,
  'warnflag': 2})